# Preamble

## Variables

In [1]:
DATA_AMOUNT = [10, 100]
TRAIN_SPLIT = 0.8

## Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import get_text
import ceaser

## Creating Data

In [4]:
trainData, testData = get_text.initialise(ceaser.encrypt, *DATA_AMOUNT, TRAIN_SPLIT)

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
trainData

[['vjg rtqlgev iwvgpdgti gdqqm cffkuqp da yknnkco lqjp eqwtvjqrg vjku gdqqm ku hqt vjg wug qh cpaqpg cpayjgtg cv pq equv cpf ykvj cnoquv pq tguvtkevkqpu yjcvuqgxgt aqw oca eqra kv ikxg kv cyca qt tg wug kv wpfgt vjg vgtou qh vjg rtqlgev iwvgpdgti nkegpug kpenwfgf ykvj vjku gdqqm qt qpnkpg cv yyy iwvgpdgti qti vkvng cffkuqp cwvjqt yknnkco lqjp eqwtvjqrg tgngcug fcvg pqxgodgt gdqqm ncpiwcig gpinkuj ejctcevgt ugv gpeqfkpi kuq uvctv qh vjg rtqlgev iwvgpdgti gdqqm cffkuqp g vgzv rtgrctgf da vjg qpnkpg fkuvtkdwvgf rtqqhtgcfkpi vgco jvvr yyy rifr pgv htqo rcig kocigu igpgtqwuna ocfg cxckncdng da kpvgtpgv',
  2],
 ['ifudiuh h m sxkhsx jxqsauhqo qdjxedo jhebbefu rkhau zexd cehbuo cybjed cqha fqjjyied xqmjxehdu xudho zqcui zh iekjxuo u temtud sxqksuh q m mqht rkdoqd z q vhektu semfuh webtmyd icyjx fefu buibyu ijufxud rohed zexd dysxeb besau jxecqi vembuh mehtimehjx v couhi thotud w iqydjirkho bqdteh iytduo seblyd tu gkydsuo tqlyt cqiied bqcr qbvhut qydwuh rudjbuo h s zurr tysaudi q m mqht whqo u